## Creating the engine to connect to the database "sakila"

In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [ ]:
db = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+ db
engine = create_engine(connection_string)
engine

## Python function "rentals_month"

In [ ]:
def rentals_month(month, year):
    ''' Function that retrieves rental data for a given month and year 
    (passed as parameters) from the Sakila database.
    Month as integer, Year as integer (%Y).
    Returns a Pandas DataFrame.'''

    # converting it in strings, and formatting month always with 2 digids
    year = str(year)
    month = (lambda month: str(month) if len(str(month)) == 2 else '0'+str(month))(month)
             
    with engine.connect() as connection:
        query = text(f'''SELECT * FROM rental
                        WHERE DATE_FORMAT(CONVERT(rental_date, DATE), '%m') = {month} 
                        AND DATE_FORMAT(CONVERT(rental_date, DATE), '%Y') = {year};''')
        result = connection.execute(query)
        df = pd.DataFrame(result.all())

    return df  

## Python function "rental_count_month"

In [ ]:
def rental_count_month(df, month, year):
    ''' Function that takes the DataFrame provided by rentals_month as input along with 
    the month and year and returns a new DataFrame containing the number of rentals made 
    by each customer_id during the selected month and year.'''

    # converting it in strings, and formatting month always with 2 digids
    year = str(year)
    month = (lambda month: str(month) if len(str(month)) == 2 else '0'+str(month))(month)

    new_df = df.groupby('customer_id')['rental_id'].count().reset_index()
    new_df.rename(columns = {'rental_id': 'rentals_{}_{}'.format(month,year)}, inplace = True)

    return new_df

## Python function "compare_rentals"

In [ ]:
def compare_rentals(df1, df2):
    '''Function that takes two DataFrames as input containing the number of rentals 
    made by each customer in different months and years.
    Returns a combined DataFrame with a new 'difference' column, which is the difference 
    between the number of rentals in the two months.'''

    new_df = pd.merge(df1, df2, how = 'outer', on = 'customer_id')

    new_df["difference"] = df.iloc[:, 2] - df.iloc[:, 1]

    return new_df 